## XỬ LÍ - LÀM SẠCH DỮ LIỆU


#### I. Mô tả dữ liệu 

In [1]:
import pandas as pd # Dùng để xử lý dữ liệu dạng bảng
import numpy as np # Dùng để xử lý dữ liệu dạng mảng

from imblearn.over_sampling import SMOTE # Dùng để cân bằng dữ liệu

from sklearn.model_selection import train_test_split # Dùng để chia dữ liệu thành tập huấn luyện và tập kiểm tra
from sklearn.preprocessing import StandardScaler # Dùng để chuẩn hóa dữ liệu

In [2]:
# Đọc dữ liệu từ file csv
stroke_prediction_df = pd.read_csv('File dữ liệu thô.csv')
stroke_prediction_df.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [3]:
# Kiểm tra thông tin sơ bộ của Data Frame
stroke_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


Dữ liệu được lấy từ bộ dữ liệu chăm sóc sức khỏe (Healthcare Dataset - Stroke Data), gồm nhiều thông tin liên quan đến hồ sơ y tế của bệnh nhân.
Một số cột quan trọng trong tập dữ liệu gồm:

- id: Mã định danh bệnh nhân (không dùng trong dự đoán).
- gender: Giới tính.
- age: Tuổi.
- hypertension: Tình trạng huyết áp cao.
- heart_disease: Bệnh tim.
- ever_married: Tình trạng hôn nhân.
- work_type: Loại công việc.
- Residence_type: Khu vực sống (nông thôn/thành thị).
- avg_glucose_level: Mức đường huyết trung bình.
- bmi: Chỉ số khối cơ thể.
- smoking_status: Tình trạng hút thuốc.
stroke: Cột mục tiêu (1 nếu có đột quỵ, 0 nếu không).

#### II. Thực hiện tiền xử lí

1. Xử lí dữ liệu không liên quan


- Trong bộ dữ liệu, cột id chỉ đóng vai trò là mã số định danh cho từng bản ghi (record). Nó không mang thông tin thực chất nào về đặc điểm của bệnh nhân hoặc kết quả bệnh lý, nên không có giá trị dự đoán trong mô hình học máy.

In [4]:
# Xử lý cột id và loại bỏ bản ghi trùng lặp
stroke_prediction_df.drop(columns='id', inplace=True)
stroke_prediction_df.drop_duplicates(inplace=True)


2. Chia dữ liệu (lấy 80% dữ liệu để train, 20% dữ liệu để test)

In [5]:
# Tách dữ liệu
X = stroke_prediction_df.drop('stroke', axis=1)
y = stroke_prediction_df['stroke']

# Chia dữ liệu, lấy 20% dữ liệu để làm test, 80% dữ liệu để train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Tạo bản sao (copy) rõ ràng để tránh lỗi SettingWithCopyWarning
X_train = X_train.copy()
X_test = X_test.copy()

- Mục đích của việc chia dữ liệu thành tập huấn luyện (train set) và tập kiểm tra (test set) là để đánh giá hiệu suất của mô hình học máy một cách khách quan. Tập huấn luyện được sử dụng để huấn luyện mô hình, trong khi tập kiểm tra được giữ riêng biệt để kiểm tra khả năng tổng quát hóa của mô hình trên dữ liệu chưa từng thấy trước đó. Việc này giúp đảm bảo rằng mô hình không chỉ học thuộc lòng dữ liệu huấn luyện mà còn có thể áp dụng tốt trên dữ liệu thực tế.

3. Xử lý Dữ liệu thiếu

In [6]:
# 1. Học median từ X_train
bmi_median_train = X_train['bmi'].median()

# 2. Áp dụng cho CẢ X_train và X_test (gán giá trị trực tiếp)
X_train['bmi'] = X_train['bmi'].fillna(bmi_median_train)
X_test['bmi'] = X_test['bmi'].fillna(bmi_median_train)

- Vì BMI là dữ liệu số, thường phân bố lệch nhẹ, nên median (trung vị) sẽ an toàn hơn mean (trung bình) khi có outlier.
- Trước hết, giá trị trung vị (median) của bmi được tính chỉ trên tập huấn luyện (X_train) để tránh rò rỉ dữ liệu (data leakage). Sau đó, giá trị trung vị này được dùng để thay thế các giá trị NaN trong cả X_train và X_test thông qua phương thức fillna(). Cách làm này giúp đảm bảo mô hình được huấn luyện và kiểm tra trên dữ liệu đã được xử lý nhất quán, đồng thời không sử dụng thông tin từ tập kiểm tra trong quá trình huấn luyện.

4. Xử lý giá trị ngoại lai (Outlier)

In [7]:
# Chọn các cột numeric
numeric_cols = ['age', 'avg_glucose_level', 'bmi']

# Tính Q1, Q3 và IQR trên tập Train
Q1 = X_train[numeric_cols].quantile(0.25)
Q3 = X_train[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Giới hạn giá trị bằng clip theo cột (axis=1) cho cả tập train và test
X_train[numeric_cols] = X_train[numeric_cols].clip(lower=Q1 - 1.5 * IQR,
                                                   upper=Q3 + 1.5 * IQR,
                                                   axis=1)
X_test[numeric_cols] = X_test[numeric_cols].clip(lower=Q1 - 1.5 * IQR,
                                                 upper=Q3 + 1.5 * IQR,
                                                 axis=1)

2. Mã hóa dữ liệu (Encoding Categorical Data):

In [8]:
# Các cột cần mã hóa
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Áp dụng one-hot encoding cho tập train và test
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Đồng bộ cột giữa train và test
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

print("Kích thước tập huấn luyện sau khi xử lý:", X_train.shape)
print("Kích thước tập kiểm tra sau khi xử lý:", X_test.shape)


Kích thước tập huấn luyện sau khi xử lý: (4088, 16)
Kích thước tập kiểm tra sau khi xử lý: (1022, 16)


- Áp dụng phương pháp One-Hot Encoding để chuyển các giá trị phân loại thành các biến nhị phân (0 và 1). Mỗi giá trị trong một cột được mã hóa thành một cột mới đại diện riêng biệt.
Việc này giúp mô hình hiểu đúng bản chất của dữ liệu, tránh việc hiểu sai thứ tự giữa các giá trị (như khi dùng Label Encoding), đồng thời cải thiện khả năng học và dự đoán của mô hình.

#### III. Chuẩn hóa và cân bằng dữ liệu

1. Chuẩn hóa dữ liệu

In [9]:
numeric_cols = ['age', 'avg_glucose_level', 'bmi']

# Chuẩn hóa dữ liệu numeric với StandardScaler
scaler = StandardScaler()

# Áp dụng chuẩn hóa cho cả tập train và test
# Fit và transform trên X_train
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
# Chỉ transform trên X_test (dùng scaler đã fit từ train)
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])


# Gán tên lại (DataFrame X_train lúc này đã là cái ta cần)
X_train_prepared = X_train
X_test_prepared = X_test

print("Đã chuẩn hóa.")
print("Shape X_train_prepared:", X_train_prepared.shape)
print("Shape X_test_prepared:", X_test_prepared.shape)
X_train_prepared.head(10)

Đã chuẩn hóa.
Shape X_train_prepared: (4088, 16)
Shape X_test_prepared: (1022, 16)


,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
845,0.205661,0,0,-0.960346,0.616266,False,False,True,False,True,False,False,True,False,True,False
3744,-1.254901,0,0,0.632522,-1.079182,True,False,False,False,True,False,False,False,False,True,False
4183,1.046590,0,0,0.277250,-0.532715,False,False,True,False,False,True,False,False,False,True,False
3409,0.028623,0,0,-1.074466,-0.546727,True,False,True,False,True,False,False,True,False,False,True
284,-1.299160,0,0,-0.566033,0.406086,True,False,False,False,False,False,False,True,False,False,False
4796,1.312147,1,0,-0.897881,1.260816,True,False,False,False,False,False,False,True,False,True,False
1283,0.869552,1,1,1.479409,2.213630,True,False,True,False,True,False,False,True,True,False,False
3656,-0.059896,0,0,-0.776253,1.302852,False,False,True,False,False,True,False,False,False,True,False
2485,-1.299160,0,0,-0.920404,-0.602775,False,False,False,False,True,False,False,False,True,False,False
1282,0.426958,0,1,2.052258,0.462134,True,False,True,False,True,False,False,False,False,False,False


- Việc chuẩn hóa các cột numeric như age, avg_glucose_level và bmi là bước quan trọng trong tiền xử lý dữ liệu, giúp đưa tất cả các biến về cùng thang đo với trung bình gần 0 và độ lệch chuẩn bằng 1. Điều này rất cần thiết vì các biến ban đầu có phạm vi và đơn vị khác nhau, nếu không chuẩn hóa, những biến có giá trị lớn sẽ chi phối mô hình, làm thuật toán học máy nhạy cảm với các giá trị này.

2. Cân bằng dữ liệu bằng phương pháp SMOTE (Synthetic Minority Oversampling Technique)

- Trong tập dữ liệu này, số lượng người không bị đột quỵ (stroke = 0) nhiều hơn rất nhiều so với người bị đột quỵ (stroke = 1).
Nếu không cân bằng, mô hình sẽ có xu hướng dự đoán nghiêng về lớp chiếm đa số, dẫn đến độ chính xác ảo nhưng không phát hiện được các trường hợp đột quỵ thật.

In [10]:
# Sử dụng SMOTE để cân bằng dữ liệu
smote = SMOTE(random_state=42)

# Áp dụng SMOTE chỉ trên tập huấn luyện
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_prepared, y_train)
print("Dữ liệu sau khi cân bằng:")
print("Shape X_train_resampled:", X_train_resampled.shape)
print("Shape y_train_resampled:", y_train_resampled.shape)


Dữ liệu sau khi cân bằng:
Shape X_train_resampled: (7778, 16)
Shape y_train_resampled: (7778,)


- Chú ý: Trong quá trình xây dựng mô hình học máy, việc cân bằng dữ liệu chỉ nên được thực hiện trên tập huấn luyện (train set) mà không áp dụng cho tập kiểm tra (test set). Lý do là vì mục đích của cân bằng dữ liệu là giúp mô hình học tốt hơn khi các lớp bị mất cân đối, ví dụ như trong bài toán dự đoán đột quỵ, số lượng người “có đột quỵ” rất ít so với “không đột quỵ”. Tuy nhiên, tập kiểm tra được dùng để đánh giá khả năng tổng quát hóa của mô hình trên dữ liệu thực tế, nên cần được giữ nguyên phân bố ban đầu. Nếu ta thực hiện cân bằng trên tập test, dữ liệu kiểm tra sẽ bị thay đổi, gây ra hiện tượng “rò rỉ dữ liệu” (data leakage) và khiến kết quả đánh giá không phản ánh đúng hiệu suất mô hình trong thực tế. Do đó, quy trình đúng là chỉ cân bằng dữ liệu trên tập train để huấn luyện mô hình, còn tập test phải được giữ nguyên để đảm bảo việc đánh giá khách quan và chính xác.

In [11]:

# Gộp dữ liệu lại thành Data Frame để thực hiện thống kê mô tả, dễ quan sát
stroke_prediction_train = pd.DataFrame(X_train_resampled, columns=X_train_prepared.columns)
stroke_prediction_train['stroke'] = y_train_resampled


In [12]:
stroke_prediction_train['stroke'].value_counts()

stroke
0    3889
1    3889
Name: count, dtype: int64

3. Thống kê mô tả (Dựa trên dữ liệu cân bằng của tập train)

In [13]:
stroke_prediction_train.describe()


,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,7778.000000,7778.000000,7778.000000,7778.000000,7778.000000,7778.000000
mean,0.516884,0.119054,0.071098,0.239198,0.083229,0.500000
std,0.958480,0.323873,0.257005,1.161115,0.872805,0.500032
min,-1.915251,0.000000,0.000000,-1.383489,-2.578462,0.000000
25%,-0.015636,0.000000,0.000000,-0.694133,-0.420707,0.000000
50%,0.734070,0.000000,0.000000,-0.165592,-0.056308,0.500000
75%,1.312147,0.000000,0.000000,1.572625,0.518182,1.000000
max,1.710482,1.000000,1.000000,2.052258,2.465845,1.000000


In [14]:

stroke_prediction_train.to_csv('Dữ liệu sau tiền xử lý1111.csv', index=False)

Dựa trên bảng thống kê mô tả, có thể nhận xét như sau:

Bộ dữ liệu sau khi được chuẩn hóa có tổng cộng 7.778 quan sát, với các biến liên quan đến tuổi (age), bệnh cao huyết áp (hypertension), bệnh tim (heart_disease), mức đường huyết trung bình (avg_glucose_level), chỉ số khối cơ thể (bmi) và biến mục tiêu (stroke). Các giá trị trung bình (mean) của các biến hypertension, heart_disease và stroke lần lượt là 0.12, 0.08 và 0.50, cho thấy tỷ lệ người mắc cao huyết áp và bệnh tim trong dữ liệu là tương đối thấp, trong khi dữ liệu đã được cân bằng về biến mục tiêu nên tỷ lệ người bị đột quỵ là 50%. Các biến định lượng như age, avg_glucose_level và bmi có giá trị trung bình lần lượt là 0.51, 0.24 và 0.09 sau khi chuẩn hóa, thể hiện mức phân bố xung quanh trung tâm (0) và độ lệch chuẩn khoảng 1 — đây là đặc trưng của dữ liệu đã được chuẩn hóa bằng StandardScaler.

Ngoài ra, giá trị nhỏ nhất và lớn nhất của các biến chuẩn hóa lần lượt dao động trong khoảng từ -2.5 đến 2.4, cho thấy dữ liệu không có sự lệch chuẩn quá lớn hay ngoại lai nghiêm trọng sau khi xử lý. Nhìn chung, bộ dữ liệu có phân bố khá cân đối sau khi chuẩn hóa và cân bằng, đảm bảo điều kiện phù hợp để huấn luyện mô hình học máy.

####  IV. Tổng kết quá trình tiền xử lý dữ liệu


Quá trình tiền xử lý dữ liệu cho bài toán dự đoán đột quỵ đã được thực hiện qua các bước quan trọng như sau:
1. Xử lý dữ liệu không liên quan: Loại bỏ cột id không mang thông tin dự đoán.
2. Chia dữ liệu: Tách dữ liệu thành tập huấn luyện (80%) và tập kiểm tra (20%).
3. Xử lý dữ liệu thiếu: Thay thế giá trị thiếu ở cột bmi
    bằng giá trị trung vị (median) tính từ tập huấn luyện.
4. Xử lý giá trị ngoại lai (outlier): Sử dụng phương pháp cắt tỉa (clipping) để giới hạn các giá trị bất thường trong các cột numeric.
5. Mã hóa dữ liệu phân loại (categorical data): Áp dụng One-Hot Encoding để chuyển đổi các biến phân loại thành các biến nhị phân.
6. Chuẩn hóa dữ liệu: Sử dụng StandardScaler để chuẩn hóa các biến numeric về cùng thang đo.
7. Cân bằng dữ liệu: Áp dụng SMOTE chỉ trên tập huấn luyện để tạo ra một tập dữ liệu cân bằng giữa các lớp mục tiêu.
8. Thống kê mô tả: Phân tích và đánh giá đặc điểm của bộ dữ liệu sau khi tiền xử lý.
Quá trình này giúp đảm bảo dữ liệu đầu vào cho mô hình học máy là sạch,đầy đủ và phù hợp, từ đó nâng cao hiệu suất và độ chính xác của mô hình trong việc dự đoán đột quỵ. 

Các biến kết quả sau quá trình tiền xử lý để sử dụng trong mô hình học máy bao gồm:
- `X_train_resampled`: (3889+3889=7778 mẫu) Dữ liệu đặc trưng đã được cân bằng sử dụng SMOTE, bao gồm cả các biến numeric đã được chuẩn hóa và các biến categorical đã được mã hóa. 
- `y_train_resampled`: (3889+3889=7778 mẫu) Nhãn mục tiêu tương ứng với `X_train_resampled`, đã được cân bằng để có số lượng mẫu đồng đều giữa các lớp.
- `X_test_prepared`: Dữ liệu đặc trưng của tập kiểm tra, bao gồm cả các biến numeric đã được chuẩn hóa và các biến categorical đã được mã hóa.
- `y_test`: Nhãn mục tiêu tương ứng với `X_test_prepared`. 
